### Web Scraping fbref.com Premier League Data

Introduction:

In this project, we will be web scraping data from fbref.com, a website that provides statistics for football (soccer) matches and players. Our focus will be on the English Premier League, and we will be extracting team stats, individual player stats, and lineups for each game.

**Warning** Web scraping can be a powerful tool for collecting and analyzing data from the internet, but it is important to be aware of the potential risks and errors that can arise. Some possible issues to watch out for include:

* Changes to the website's structure or layout: The HTML code and structure of a website can change over time, which can break the XPath expressions or CSS selectors that we use to locate the data we want to extract.

* Blocked or rate-limited requests: Some websites may block or rate-limit requests from web scrapers to prevent overloading their servers. In these cases, we may need to use techniques such as rotating IP addresses or adding delays to our requests.

* Incorrect or incomplete data: Even if our web scraper is able to successfully extract data from a website, there is always the possibility that the data itself is incorrect or incomplete. It is important to validate and verify the data we collect to ensure its accuracy.

With these considerations in mind, let's begin our web scraping journey to collect Premier League data from fbref.com!

**Author**: Ángel Daniel Gil Contreras*

In [1]:
import requests
from scrapy import Selector
import pandas as pd
import time

#### 1.  First we need to create some usefull functions for our scraping process

In [2]:
def get_fbref_urls(url,XPath, i_url = "https://fbref.com"):
    """
    This function retrieves a list of URLs from a given URL using the specified XPath.
    The i_url parameter is used as a prefix for each URL in the list.
    
    Parameters:
        url (str): The URL from which to retrieve the list of URLs.
        XPath (str): The XPath to use for retrieving the list of URLs.
        i_url (str): A prefix to add to each URL in the list (defaults to "https://fbref.com").
        
    Returns:
        list: A list of URLs with the prefix i_url added to each one.
        
    Author: Ángel Daniel Gil Contreras
    Last updated: 2022-12-29
    """
    # Retrieve the HTML content from the URL
    html = requests.get(url).content
    
    # Use the Selector class to parse the HTML
    sel = Selector(text = html)
    
    # Use the XPath to extract a list of URLs from the HTML
    urls = set(sel.xpath(XPath).extract())
    
    # Add the i_url prefix to each URL in the list
    urls = [i_url + u for u in list(urls)]
    
    return urls


In [3]:
def get_table(url, match):
    """
    This function retrieves a table from a given URL using the specified match criteria.
    
    Parameters:
        url (str): The URL from which to retrieve the table.
        match (str): The match criteria to use for retrieving the table.
        
    Returns:
        pandas.DataFrame: The table retrieved from the URL.
        
    Author: Ángel Daniel Gil Contreras
    Last updated: 2022-12-29
    """
    # Retrieve the HTML content from the URL
    html = requests.get(url).content
    
    # Try to delete extra levels and get the table
    try:
        table = pd.read_html(html, match = match)[0]
        table.columns = table.columns.droplevel()
    except:
        table = pd.read_html(html, match = match)[0]
        
    return table


### 2. Getting Team Stats from its games

In [10]:
def get_team_stats(team_url, team_name, year = 2023):
    """
    This function retrieves possession and shooting stats for a given team from the specified URL.
    
    Parameters:
        team_url (str): The URL from which to retrieve the stats.
        team_name (str): The name of the team for which to retrieve the stats.
        year (int): The year for which to retrieve the stats (defaults to 2023).
        
    Returns:
        tuple: A tuple containing two pandas.DataFrame objects: the possession stats table and the shooting stats table.
        
    Author: Ángel Daniel Gil Contreras
    Last updated: 2022-12-29
    """
    # Print a message indicating which team and URL are being processed
    print('\n {year}: {team}: {url}'.format(year = year ,team = team_name, url = team_url))
    
    # Define the XPath for extracting URLs from the team URL
    XPath = '//*[@id="content"]//div[6]//div/a/@href'
    
    # Use the get_fbref_urls function to retrieve a list of URLs from the team URL
    team_tables_urls = get_fbref_urls(team_url,XPath)
    
    for team_table_url in team_tables_urls: 
        # Sleep for 1 second to avoid overloading the server
        time.sleep(1)
        
        # Extract the name of the table from the URL
        team_table_name = team_table_url.split('/')[-2]
        
        # If the table is the possession stats table, retrieve it and add team and year columns to it
        if team_table_name == 'possession':
                print('  {team}: possesion table'.format(team = team_name))
                pos_table = get_table(team_table_url, match = 'Posesión del balón')
                pos_table['team'] = team_name
                pos_table['year'] = year
                # Assert that the table has more than one row
                assert len(pos_table) > 1
                print('   Pos table Status: Done!!')
        
        # If the table is the shooting stats table, retrieve it and add team and year columns to it
        if team_table_name == 'shooting':
                print('  {team} shooting  table'.format(team = team_name))
                shot_table = get_table(team_table_url, match = 'Tiros')
                shot_table['team'] = team_name
                shot_table['year'] = year
                # Assert that the table has more than one row
                assert len(shot_table) > 1
                print('   Shot table Status: Done!! \n')
                
    # Return the possession stats table and the shooting stats table as a tuple
    return pos_table, shot_table


#####  Lets get the most recent 3 years stats
      
This code defines a list of years for which to retrieve data and an initial URL for the Premier League statistics page on fbref.com. It then defines an XPath for extracting the URL of the next page of statistics and initializes an empty list to store the URLs of the statistics pages. The code then iterates over the years, adding the current URL to the list of URLs and using the get_fbref_urls function to retrieve the URL of the next page of statistics.

In [8]:
# Define the years for which to retrieve data
years = [2023,2022,2021]

# Set the initial URL for the Premier League statistics page
act_temp_url = 'https://fbref.com/es/comps/9/Estadisticas-de-Premier-League'

# Define the XPath for extracting the URL of the next page of statistics
XPath = '//*[@id="meta"]/div[2]/div/a[1]/@href'

# Initialize an empty list to store the URLs of the statistics pages
temp_urls = []

# Iterate over the years
for year in years:
    # Add the current URL to the list of URLs
    temp_urls.append(act_temp_url)
    
    # Use the get_fbref_urls function to retrieve the URL of the next page of statistics
    act_temp_url = get_fbref_urls(act_temp_url,XPath, i_url = "https://fbref.com")[0]

# Print the list of URLs
print(temp_urls)


['https://fbref.com/es/comps/9/Estadisticas-de-Premier-League', 'https://fbref.com/es/comps/9/2021-2022/Estadisticas-2021-2022-Premier-League', 'https://fbref.com/es/comps/9/2020-2021/Estadisticas-2020-2021-Premier-League']


In [11]:
# This code retrieves possession and shooting stats for each team in the Premier League for the specified years from fbref.com.
# If an error occurs when retrieving the stats for a team, its URL is added to the teams_stats_error_urls list.
# The resulting possession and shooting stats tables are saved to csv files.

pos_list = []
shot_list = []
teams_stats_error_urls = []

# Iterate over the URLs for the seasons in the specified years
for i,temp_url in enumerate(temp_urls):
    time.sleep(1)
    # Extract the URLs for the team stats pages from the current season URL
    XPath = '//*[contains(@id,"results20")]/tbody//tr/td[1]/a/@href'
    teams_urls = get_fbref_urls(temp_url,XPath, i_url = "https://fbref.com")

    # Iterate over the team URLs and retrieve the possession and shooting stats tables
    for team_url in teams_urls:
        time.sleep(1)
        # Extract the team name from the URL
        team_name = team_url.split('/')[-1].replace('Estadisticas-de-', '').replace('-',' ')
        
        try:
            # Retrieve the possession and shooting stats tables for the current team
            pos, shot = get_team_stats(team_url, team_name, year = years[i])
            # Add the tables to the list of tables for all teams
            pos_list.append(pos)
            shot_list.append(shot)
            
        except:
            # If an error occurs, add the team URL to the list of error URLs and skip to the next team
            teams_stats_error_urls.append(team_url)
            next

# Concatenate the tables for all teams into a single table and save it to a csv file
pos_df = pd.concat(pos_list)
pos_df.to_csv('posesion.csv')
shot_df = pd.concat(shot_list)
shot_df.to_csv('shot.csv')
# Print the list of URLs for teams for which an error occurred when retrieving the stats
print(teams_stats_error_urls)



 2023: Manchester City: https://fbref.com/es/equipos/b8fd03ef/Estadisticas-de-Manchester-City
  Manchester City shooting  table
   Shot table Status: Done!! 

  Manchester City: possesion table
   Pos table Status: Done!!

 2023: Everton: https://fbref.com/es/equipos/d3fd31cc/Estadisticas-de-Everton
  Everton: possesion table
   Pos table Status: Done!!
  Everton shooting  table
   Shot table Status: Done!! 


 2022: Aston Villa: https://fbref.com/es/equipos/8602292d/2021-2022/Estadisticas-de-Aston-Villa
  Aston Villa: possesion table
   Pos table Status: Done!!
  Aston Villa shooting  table
   Shot table Status: Done!! 


 2022: Watford: https://fbref.com/es/equipos/2abfe087/2021-2022/Estadisticas-de-Watford
  Watford shooting  table
   Shot table Status: Done!! 

  Watford: possesion table
   Pos table Status: Done!!
[]


#####     Test

In [13]:
print(pos_df['year'].unique())
print(pos_df['team'].unique())
pos_df.head()

[2023 2022]
['Manchester City' 'Everton' 'Aston Villa' 'Watford']


,Fecha,Hora,Comp,Ronda,Día,Sedes,Resultado,GF,GC,Adversario,...,Con éxito,Int.,% de éxito,Errores de control,Des,Rec,Prog,Informe del partido,team,year
0,2022-07-30,17:00,Community Shield,FA Community Shield,Sáb,Neutral,D,1,3,Liverpool,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Informe del partido,Manchester City,2023
1,2022-08-07,16:30,Premier League,Semana 1 de partido,Dom,Visitante,V,2,0,West Ham,...,14.0,26.0,53.8,7.0,3.0,782.0,34.0,Informe del partido,Manchester City,2023
2,2022-08-13,15:00,Premier League,Semana 2 de partido,Sáb,Local,V,4,0,Bournemouth,...,8.0,14.0,57.1,10.0,6.0,664.0,36.0,Informe del partido,Manchester City,2023
3,2022-08-21,16:30,Premier League,Semana 3 de partido,Dom,Visitante,E,3,3,Newcastle Utd,...,13.0,23.0,56.5,9.0,6.0,546.0,40.0,Informe del partido,Manchester City,2023
4,2022-08-27,15:00,Premier League,Semana 4 de partido,Sáb,Local,V,4,2,Crystal Palace,...,6.0,15.0,40.0,10.0,5.0,731.0,61.0,Informe del partido,Manchester City,2023


### 3. Getting Players Stats from its games

    We will need to define the next function:

In [18]:
def get_player_stats(player_url, player_name, team_name, year = 2023):  
    """
    This function retrieves match logs for a given player from the specified URL.
    
    Parameters:
        player_url (str): The URL from which to retrieve the stats.
        player_name (str): The name of the player for which to retrieve the stats.
        team_name (str): The name of the team the player belongs to.
        year (int): The year for which to retrieve the stats (defaults to 2023).
        
    Returns:
        pandas.DataFrame: A table containing the match logs for the player.
        
    Author: Ángel Daniel Gil Contreras
    Date: 2022-12-29
    """
    # Modify the URL to point to the match logs section
    url = player_url + '#matchlogs_for'
    
    # Print a message indicating which player and URL are being processed
    print('    {year}: {team_name}: {player_name}: {url}'.format(year = year, team_name = team_name, player_name = player_name, url = url))
    
    # Retrieve the match logs table from the URL
    match = "Registros de partidos"
    table = get_table(url,match)
    
    # Add player, team, and year columns to the table
    table['name'] = player_name
    table['team'] = team_name
    table['year'] = year
    
    # Print a success message
    print('\tStatus: éxito!!\n')
    
    # Return the table
    return table


The next code retrieves individual player stats for the Premier League from fbref.com.
It loops through a list of URLs for each season and team, and then retrieves the URLs for each player on the team.
It then calls the get_player_stats function to retrieve the player stats table for each player, and appends it to a list.
Finally, it concatenates all of the player stats tables into a single pandas DataFrame and saves it to a CSV file.

In [21]:
# Author: Ángel Daniel Gil Contreras
# Date: 2022-12-29

players_list = []
players_error_urls = []

# Loop through the list of URLs for each season
for i, temp_url in enumerate(temp_urls):
    # Retrieve the URLs for each team in the season
    XPath = '//*[contains(@id,"results20")]/tbody//tr/td[1]/a/@href'
    teams_urls = get_fbref_urls(temp_url,XPath, i_url = "https://fbref.com")

    # Loop through the list of URLs for each team
    for team_url in teams_urls:
        # Sleep for 1 second to avoid overloading the server
        time.sleep(1)
        
        # Get the team name from the URL
        team_name = team_url.split('/')[-1].replace('Estadisticas-de-', '').replace('-',' ')
        
        # Retrieve the URLs for each player on the team
        XPath = '//*[contains(@id,"stats_standard")]/tbody//tr/td[29]//a/@href'
        players_urls = get_fbref_urls(team_url,XPath, i_url = "https://fbref.com")
        
        # Loop through the list of URLs for each player
        for player_url in players_urls:
            # Sleep for 1 second to avoid overloading the server
            time.sleep(1)
            
            # Get the player name from the URL
            player_name = player_url.split('/')[-1].replace('#matchlogs_for','').replace('-',' ').replace('Registros de partidos de ','')
            
            # Try to retrieve the player stats table and append it to the list
            try:
                player = get_player_stats(player_url, player_name, team_name, year = years[i])
                players_list.append(player)
            except:
                # If an error occurs, add the player URL to the list of error URLs
                players_error_urls.append(player_url)
                next

# Concatenate all of the player stats tables into a single pandas DataFrame and save it to a CSV file
players_df = pd.concat(players_list)
players_df.to_csv('players.csv')

print(players_error_urls)

    2023: Manchester City: Aymeric Laporte: https://fbref.com/es/jugadores/119b9a8e/partidos/2022-2023/summary/Registros-de-partidos-de-Aymeric-Laporte#matchlogs_for
	Status: éxito!!

    2023: Manchester City: Cole Palmer: https://fbref.com/es/jugadores/dc7f8a28/partidos/2022-2023/summary/Registros-de-partidos-de-Cole-Palmer#matchlogs_for
	Status: éxito!!

    2023: Everton: Seamus Coleman: https://fbref.com/es/jugadores/0420d84f/partidos/2022-2023/summary/Registros-de-partidos-de-Seamus-Coleman#matchlogs_for
	Status: éxito!!

    2023: Everton: Amadou Onana: https://fbref.com/es/jugadores/828657ff/partidos/2022-2023/summary/Registros-de-partidos-de-Amadou-Onana#matchlogs_for
	Status: éxito!!

    2022: Aston Villa: Tyrone Mings: https://fbref.com/es/jugadores/8397a50c/partidos/2021-2022/summary/Registros-de-partidos-de-Tyrone-Mings#matchlogs_for
	Status: éxito!!

    2022: Aston Villa: Kortney Hause: https://fbref.com/es/jugadores/bf8fad51/partidos/2021-2022/summary/Registros-de-part

#####     Test

In [22]:
print(players_df['year'].unique())
print(players_df['team'].unique())
players_df.head()

[2023 2022]
['Manchester City' 'Everton' 'Aston Villa' 'Watford']


,Fecha,Día,Comp,Ronda,Sedes,Resultado,Equipo,Adversario,Arranque,Posc,...,Cmp,Int.,% Cmp,Prog,Con éxito,Int.,Informe del partido,name,team,year
0,2022-10-02,Dom,Premier League,Semana 9 de partido,Local,V 6–3,Manchester City,Manchester Utd,No,CB,...,18,20,90.0,0,0,0,Informe del partido,Aymeric Laporte,Manchester City,2023
1,2022-10-05,Mié,Champions Lg,Etapa de grupo,Local,V 5–0,eng Manchester City,dk FC Copenhagen,Sí,CB,...,100,107,93.5,6,0,0,Informe del partido,Aymeric Laporte,Manchester City,2023
2,2022-10-08,Sáb,Premier League,Semana 10 de partido,Local,V 4–0,Manchester City,Southampton,No,"Estuvo en el día del partido, pero no jugó.",...,"Estuvo en el día del partido, pero no jugó.","Estuvo en el día del partido, pero no jugó.","Estuvo en el día del partido, pero no jugó.","Estuvo en el día del partido, pero no jugó.","Estuvo en el día del partido, pero no jugó.","Estuvo en el día del partido, pero no jugó.",Informe del partido,Aymeric Laporte,Manchester City,2023
3,2022-10-11,Mar,Champions Lg,Etapa de grupo,Visitante,E 0–0,eng Manchester City,dk FC Copenhagen,Sí,CB,...,65,70,92.9,0,0,0,Informe del partido,Aymeric Laporte,Manchester City,2023
4,2022-10-16,Dom,Premier League,Semana 11 de partido,Visitante,D 0–1,Manchester City,Liverpool,No,"Estuvo en el día del partido, pero no jugó.",...,"Estuvo en el día del partido, pero no jugó.","Estuvo en el día del partido, pero no jugó.","Estuvo en el día del partido, pero no jugó.","Estuvo en el día del partido, pero no jugó.","Estuvo en el día del partido, pero no jugó.","Estuvo en el día del partido, pero no jugó.",Informe del partido,Aymeric Laporte,Manchester City,2023


#### 4. Getting the lineups
    
    We will define the next function to get the lineups table

In [26]:
def get_alin(match_url, team_name):
    """
    This function retrieves the lineup for a given match from the specified URL.
    
    Parameters:
        match_url (str): The URL from which to retrieve the lineup.
        team_name (str): The name of the team for which to retrieve the lineup.
        
    Returns:
        pandas.DataFrame: A table containing the lineup for the given match.
        
    Author: Ángel Daniel Gil Contreras
    Last updated: 2022-12-29
    """
    # Retrieve the table containing the lineup from the match URL
    table = get_table(match_url, match = team_name)
    
    # Extract the date of the match from the URL
    fecha = match_url.split('-')[-5:-2]
    month, day, year = fecha
    date = day+'-'+month+'-'+year
    
    # Add a 'date' column to the table and a 'team' column containing the name of the team
    table['date'] = date
    table['team'] = team_name
    print('{team}: {date}: {url}'.format(team = team_name, date = date, url = match_url))
    return table


This code scrapes data from the website and processing it to extract information about soccer match lineups. The data is then stored in a dataframe and saved to a CSV file.

The code first defines a dictionary of team names, with the keys being the original names and the values being the modified names. It then initializes two lists: alineacion_list, which will store the data for each match lineup, and match_error_urls, which will store any URLs that cause errors during the scraping process.

The code then loops through a list of URLs stored in the temp_urls variable, using the get_fbref_urls function to extract the URLs of the team pages from each of these URLs. It then loops through these team URLs, using the get_fbref_urls function again to extract the URLs of the match pages for each team.

For each match URL, the code checks if the team name is in the names dictionary and, if it is, replaces it with the modified name. It then uses the get_alin function to extract the lineup data for the match and stores it in a dataframe. The dataframe is then modified to add a column indicating whether each player is a starter (tit = 1) or a substitute (tit = 0). The modified dataframe is then added to the alineacion_list. If an error occurs during the scraping process, the URL is added to the match_error_urls list.

Once all the matches have been processed, the dataframes in alineacion_list are concatenated into a single dataframe and saved to a CSV file. The list of URLs that caused errors is then printed.

Author: Ángel Daniel Gil Contreras, Date: 29/12/2022

In [29]:
# Define a dictionary of team names, with the keys being the original names and the values being the modified names
names = {'Manchester United': 'Manchester Utd', 'Tottenham Hotspur': 'Tottenham', 'Newcastle United':'Newcastle Utd', 
         'West Ham United':'West Ham', 'Nottingham Forest':"Nott'ham Forest", 'Brighton and Hove Albion': "Brighton",
        'Wolverhampton Wanderers':'Wolves', 'Sheffield United':'Sheffield Utd', 'West Bromwich Albion': 'West Brom'}

# Initialize two lists: one to store the data for each match lineup, and one to store any URLs that cause errors during the scraping process
alineacion_list = []
match_error_urls = []

# Loop through a list of URLs stored in the temp_urls variable
for temp_url in temp_urls:
    
    # Use the get_fbref_urls function to extract the URLs of the team pages from the current URL
    XPath = '//*[contains(@id,"results20")]/tbody//tr/td[1]/a/@href'
    teams_urls = get_fbref_urls(temp_url,XPath, i_url = "https://fbref.com")

    # Loop through the URLs of the team pages
    for team_url in teams_urls:
        
        # Extract the team name from the URL
        team_name = team_url.split('/')[-1].replace('Estadisticas-de-', '').replace('-',' ')
        
        # Use the get_fbref_urls function to extract the URLs of the match pages for the current team
        XPath = '//*[@id="matchlogs_for"]/tbody/tr/td[17]/a/@href'
        matches_urls = get_fbref_urls(team_url,XPath, i_url = "https://fbref.com")
        
        # Filter the URLs to only include those from the Premier League
        matches_urls = [url for url in matches_urls if 'Premier-League' in url]

        # Loop through the URLs of the match pages
        for match_url in matches_urls:
            
            # If the team name is in the names dictionary, replace it with the modified name
            if team_name in list(names.keys()):
                team_name = names[team_name]
            
            # Use the get_alin function to extract the lineup data for the current match
            try:
                alineacion = get_alin(match_url, team_name)
                
                # Modify the dataframe to add a column indicating whether each player is a starter or a substitute
                alineacion.columns = ['num', 'name', 'date', 'team']
                alineacion['tit'] = 0
                alineacion.loc[:10,'tit'] = 1
                
                # Add the modified dataframe to the alineacion_list
                alineacion_list.append(alineacion)
                
            # If an error occurs during the scraping process, add the URL to the match_error_url
            except:
                match_error_urls.append(match_url)
                next
                
        print('\n')
                
alineacion_df = pd.concat(alineacion_list, ignore_index = True)
alineacion_df.to_csv('alineaciones.csv')

print(match_error_urls)

Manchester City: 7-Agosto-2022: https://fbref.com/es/partidos/ece62baf/West-Ham-United-Manchester-City-Agosto-7-2022-Premier-League
Manchester City: 29-Octubre-2022: https://fbref.com/es/partidos/96b9ae4e/Leicester-City-Manchester-City-Octubre-29-2022-Premier-League


Everton: 12-Noviembre-2022: https://fbref.com/es/partidos/54f23f03/Bournemouth-Everton-Noviembre-12-2022-Premier-League
Everton: 6-Agosto-2022: https://fbref.com/es/partidos/3a917cee/Everton-Chelsea-Agosto-6-2022-Premier-League


Aston Villa: 13-Febrero-2022: https://fbref.com/es/partidos/59b88877/Newcastle-United-Aston-Villa-Febrero-13-2022-Premier-League
Aston Villa: 20-Noviembre-2021: https://fbref.com/es/partidos/88f081ac/Aston-Villa-Brighton-and-Hove-Albion-Noviembre-20-2021-Premier-League


Watford: 1-Enero-2022: https://fbref.com/es/partidos/9c831c07/Watford-Tottenham-Hotspur-Enero-1-2022-Premier-League
Watford: 23-Octubre-2021: https://fbref.com/es/partidos/6985d123/Everton-Watford-Octubre-23-2021-Premier-League



#####     Test

In [31]:
print(alineacion_df['team'].unique())
print(alineacion_df['date'].unique())
alineacion_df.head(15)

['Manchester City' 'Everton' 'Aston Villa' 'Watford']
['7-Agosto-2022' '29-Octubre-2022' '12-Noviembre-2022' '6-Agosto-2022'
 '13-Febrero-2022' '20-Noviembre-2021' '1-Enero-2022' '23-Octubre-2021']


,num,name,date,team,tit
0,31,Ederson,7-Agosto-2022,Manchester City,1
1,2,Kyle Walker,7-Agosto-2022,Manchester City,1
2,3,Rúben Dias,7-Agosto-2022,Manchester City,1
3,6,Nathan Aké,7-Agosto-2022,Manchester City,1
4,7,João Cancelo,7-Agosto-2022,Manchester City,1
5,8,İlkay Gündoğan,7-Agosto-2022,Manchester City,1
6,9,Erling Haaland,7-Agosto-2022,Manchester City,1
7,10,Jack Grealish,7-Agosto-2022,Manchester City,1
8,16,Rodri,7-Agosto-2022,Manchester City,1
9,17,Kevin De Bruyne,7-Agosto-2022,Manchester City,1


In [32]:
alineacion_df.tail(15)

,num,name,date,team,tit
153,18,Ozan Tufan,23-Octubre-2021,Watford,1
154,19,Moussa Sissoko,23-Octubre-2021,Watford,1
155,23,Ismaila Sarr,23-Octubre-2021,Watford,1
156,29,Cucho,23-Octubre-2021,Watford,1
157,33,Juraj Kucka,23-Octubre-2021,Watford,1
158,Banquillo,Banquillo,23-Octubre-2021,Watford,0
159,26,Daniel Bachmann,23-Octubre-2021,Watford,0
160,6,Imran Louza,23-Octubre-2021,Watford,0
161,8,Tom Cleverley,23-Octubre-2021,Watford,0
162,10,João Pedro,23-Octubre-2021,Watford,0
